# Läsionsmaske erstellen

In [ ]:
import os
import sys
import numpy as np
import scipy.io as sio

In [ ]:
data = np.load('../datasets/Tumor_2/mask.npy')#np.load('MetabMaps/Tumor_1_denoised_Fabian/Glx_amp_Tumor_1_denoised_Fabian_Orig.npy')[...,-1]
#data = data.swapaxes(0,2)

mask = np.load('../datasets/Tumor_2/tumor_mask.npy')

In [ ]:
import matplotlib.pyplot as plt

nz = 21
fig, axes = plt.subplots(nz, 2, figsize=(10, 80))  # 21 Zeilen, 2 Spalten

for z in range(nz):
    # linke Spalte: Daten
    ax1 = axes[z, 0]
    ax1.imshow(mask[:, :, z], cmap="gray", origin="lower")
    ax1.set_title(f"Data z={z}")
    ax1.axis("off")

    # rechte Spalte: Maske
    ax2 = axes[z, 1]
    ax2.imshow(data[:, :, z], cmap="gray", origin="lower")
    ax2.set_title(f"Mask z={z}")
    ax2.axis("off")

plt.tight_layout()
plt.show()



### Neue Datenstruktur nur mit Tumor Maske erstellen

In [ ]:
# neuen array nu rmit lesion voxel erstellen

import numpy as np
import os

data = np.load('../datasets/Tumor_2_normalized/data.npy')
mask = np.load('../datasets/Tumor_2_normalized/tumor_mask.npy')

# data: (X,Y,Z,t,T), mask: (X,Y,Z) mit 1 innerhalb der Läsion
X, Y, Z, t, T = data.shape

# Indizes aller Läsions-Voxel
lesion_idx = np.argwhere(mask == 1)
if lesion_idx.size == 0:
    raise ValueError("Maske enthält keine Läsions-Voxel (==1).")

# Für jede räumliche Position einen zufälligen Läsions-Index ziehen (mit Zurücklegen)
sel = np.random.randint(0, len(lesion_idx), size=(X, Y, Z))

# Quellkoordinaten (je (X,Y,Z))
src_x = lesion_idx[sel, 0]
src_y = lesion_idx[sel, 1]
src_z = lesion_idx[sel, 2]

# Zusammenbauen: jedes Zielvoxel bekommt die (t,T)-Daten seines zufällig gewählten Läsionsvoxels
lesion = data[src_x, src_y, src_z, :, :]   # Shape: (X, Y, Z, t, T)

# Ordner vorbereiten
outdir = '../datasets/Tumor_2_normalized_only_tumor_voxels'
os.makedirs(outdir, exist_ok=True)

# Speichern
#np.save(os.path.join(outdir, 'data.npy'), lesion)

lesion_ft = np.fft.fftshift(np.fft.fft(lesion, axis=3), axes=3)